In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

Creo una funcion para entrenar distintos modelos base, y luego obtener un diccionario con los mejores resultados

In [21]:
def modelo(X_train, X_test, y_train, y_test, m):
    
    if m == 'rl':
        model = LogisticRegression(max_iter = 1000, random_state = 42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        modelos['rl'] = accuracy_score(y_test, y_pred)
        return model
    elif m == 'tree':
        model = DecisionTreeClassifier(max_depth = 5, random_state = 42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        modelos['tree'] = accuracy_score(y_test, y_pred)
        return model
    elif m == 'rf':
        model = RandomForestClassifier(random_state = 42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        modelos['RF'] = accuracy_score(y_test, y_pred)
        return model
    elif m == 'svm':
        model = SVC(kernel = 'linear', random_state = 42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        modelos['svc'] = accuracy_score(y_test, y_pred)
        return model
    else:
        print("Error.")

Cargo y separo los datos en enranamiento y test

In [69]:
train, test = pd.read_csv('trainn.csv'), pd.read_csv('testt.csv')

X = train[['Pclass', 'Sex', 'Age']]
y = train.Survived.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [10]:
X_test_kaggle = test[['Pclass', 'Sex', 'Age']]

In [22]:
modelos = {}
modelos_lista = ['rl', 'tree', 'rf', 'svm']
for i in modelos_lista:
    m = modelo(X_train, X_test, y_train, y_test, i)

Visualizo resultados modelos, aparentemente el mejro resultado lo tiene el random forest, asi que procedo a utilizar grid search para buscar parametros mejores

In [23]:
modelos

{'rl': 0.7668161434977578,
 'tree': 0.7757847533632287,
 'RF': 0.7892376681614349,
 'svc': 0.7668161434977578}

In [48]:
params = {
    'n_estimators': [100, 150],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 20]
}

bosque = RandomForestClassifier()
grid_search = GridSearchCV(bosque, params, cv = 5, scoring = 'accuracy')
grid_search.fit(X_train, y_train)
print("Mejores hiperparametros:", grid_search.best_params_)

Mejores hiperparametros: {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 150}


In [65]:
bosque_n = grid_search.best_estimator_
y_pred = bosque_n.predict(X_test)
accuracy_score(y_test, y_pred)

0.7847533632286996

Genero predicciones y guardo modelo

In [66]:
predecir = bosque_n.predict(X_test_kaggle)
bosque_df = pd.DataFrame({'PassengerId': test.PassengerId,
                         'Survived': predecir
                         })
bosque_df.set_index('PassengerId', inplace = True)

In [67]:
bosque_df

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [68]:
bosque_df.to_csv('Rdo_final.csv')